# Tutorial on NashPy for Nash Equilibrium and Strategic Domination
### _Gabriele Sottocornola_ 
Game Theory, Free University of Bozen-Bolzano

In [ ]:
#Libraries import
import nashpy
import numpy as np

## Game creation

* NashPy supports both zero-sum and common payoff games with 2 players (not more).
* A game is defined by means of the two players payoff matrices, as they are represented in the combined payoff matrix
* The creation of a game support numpy arrays for the definition of the payoffs

In [ ]:
def prisoner_dilemma_game():
    a_payoff = np.array([[2, 5], [0, 4]])
    b_payoff = np.array([[2, 0], [5, 4]])
    return nashpy.Game(a_payoff, b_payoff)

In [ ]:
def modified_prisoner_dilemma_game():
    a_payoff = np.array([[2, 5], [0, 5]])
    b_payoff = np.array([[2, 0], [5, 4]])
    return nashpy.Game(a_payoff, b_payoff)

In [ ]:
def coordination_game():
    a_payoff = np.array([[2, 0], [0, 1]])
    b_payoff = np.array([[1, 0], [0, 2]])
    return nashpy.Game(a_payoff, b_payoff)

In [ ]:
def chicken_game():
    a_payoff = np.array([[0, 8], [1, 5]])
    b_payoff = np.array([[0, 1], [8, 5]])
    return nashpy.Game(a_payoff, b_payoff)

In [ ]:
def matching_pennies_game():
    zs_payoff = np.array([[1, -1], [-1, 1]])
    return nashpy.Game(zs_payoff)

In [ ]:
def rock_paper_scissor_game():
    zs_payoff = np.array([[0, -1, 1], [1, 0, -1], [-1, 1, 0]])
    return nashpy.Game(zs_payoff)

In [ ]:
def domination_elimination_game():
    a_payoff = np.array([[10, 5, 3], [0, 4, 6], [2, 3, 2]])
    b_payoff = np.array([[4, 3, 2], [1, 6, 0], [1, 5, 8]])
    return nashpy.Game(a_payoff, b_payoff)

## Utility computation

* Utilities can be computed by passing the vectors which represent the mixed strategy of each player - i.e. the distribution over the set of pure strategies for each player
* Utilities for both player are accessed as a vector index of the game object - Game(row_strategy, column_strategy) : return (row_utility, column_utility)
* No constraint in the library for inconsistent strategy definition (e.g. sum(strategy_vector) > 1)

In [ ]:
#create the game of prisoner dilemma
game = prisoner_dilemma_game()
game

In [ ]:
#both players select their first strategy (pure strategy utility)
row_strategy = [1, 0]
column_strategy = [1, 0]
utility = game[row_strategy, column_strategy]
utility

In [ ]:
#both players select a uniform random strategy (mixed strategy utility)
row_strategy = [.5, .5]
column_strategy = [.5, .5]
utility = game[row_strategy, column_strategy]
utility

## Different algorithms for Nash Equilibrium

* 3 different implementations of algorithms to find (all) the mixed Nash Equilibria of a game (Nisan et al., 2007):
    - ***Support enumeration***: Solve linear equation constraints over all possible games
    - ***Vertex enumeration***: Check for best response polytopes
    - ***Lemke Howson***: Based on integer pivoting and best response polytopes (not guarantee to find all equilibria)
* The functions return an iterator over the set of (all) equilibria
* Each equilibrium is a tuple with the pair of mixed strategies for the 2 players


In [ ]:
game = matching_pennies_game()
game

In [ ]:
#Equilibria found by Support enumeration
for eq in game.support_enumeration():
    print(eq)

In [ ]:
#Equilibria found by Vertex enumeration
for eq in game.vertex_enumeration():
    print(eq)

In [ ]:
#Equilibria found by Lemke Howson
for eq in game.lemke_howson_enumeration():
    print(eq)

## Dominant strategies

Functionalities to check if a specific strategy for a player is (strictly or weakly) dominated by other strategies

In [ ]:
def get_payoff_matrix(game, player_index):
    '''Helper function to return the payoff matrix of a given player'''
    payoff_matrix = game.payoff_matrices[player_index]
    if player_index == 0:
        return payoff_matrix
    elif player_index == 1:
        return payoff_matrix.T
    
def get_utility_player_strategy(game, player_index, strategy_index):
    '''Helper function to return the possible utilities for a player strategy'''
    payoff_matrix = get_payoff_matrix(game, player_index)
    return payoff_matrix[strategy_index]

In [ ]:
def is_dominated(strategy_payoff, payoff_matrix):
    '''
    Check if strategy_payoff is dominated by any other strategy in payoff_matrix
    Return a tuple (dominant_index, strict), where dominant_index is the index of  
    a strategy which dominates strategy_payoff and strict is a Boolean to indicate
    strict domination
    '''
    for compared_index, compared_strategy in enumerate(payoff_matrix):
        strict_vector = strategy_payoff < compared_strategy
        weak_vector = strategy_payoff <= compared_strategy
        if sum(strict_vector) == len(strategy_payoff):
            return compared_index, True
        elif (sum(weak_vector) == len(strategy_payoff)) and (sum(strict_vector) > 0):
            return compared_index, False        
    return None, False

In [ ]:
def find_dominated_strategies(game, player_index):
    '''Given a game and a player index (0 for row, 1 for column) found all the strategies which are dominated'''
    player_payoff = get_payoff_matrix(game, player_index)
    for strategy_index, strategy in enumerate(player_payoff):
        dominance = is_dominated(strategy, np.delete(player_payoff, strategy_index, 0))
        if dominance[0] is None:
            print('For player {} strategy {} is not dominated by any other strategy'
                  .format(player_index, strategy_index))
        elif dominance[1]:
            print('For player {} strategy {} is strictly dominated by strategy {}'
                  .format(player_index, strategy_index, dominance[0]))
        else:
            print('For player {} strategy {} is weakly dominated by strategy {}'
                  .format(player_index, strategy_index, dominance[0]))

In [ ]:
game = prisoner_dilemma_game()
game

In [ ]:
for player in (0,1):
    find_dominated_strategies(game, player)

In [ ]:
game = modified_prisoner_dilemma_game()
game

In [ ]:
for player in (0,1):
    find_dominated_strategies(game, player)

### Iterated elimination of dominated strategies

Algorithm to find a game solution through iterated elimination of (strictly) dominated strategies

In [ ]:
def subset_game(game, elimination_index, player_index):
    '''Helper function to reduce a game given a elimination_index strategy to delete for player_index'''
    new_row_payoff = np.delete(game.payoff_matrices[0], elimination_index, player_index)
    new_column_payoff = np.delete(game.payoff_matrices[1], elimination_index, player_index)
    return nashpy.Game(new_row_payoff, new_column_payoff)

In [ ]:
def iterated_elimination(game, strict=True):
    '''Recursive implementation of iterated elimination of dominated strategies for both strict and weak dominance'''
    if game.payoff_matrices[0].shape == (1,1):
        print('The game is in minimal form: only one action available for both players')
        return game
    else:
        for player in (0,1):
            player_payoff = get_payoff_matrix(game, player)
            for index, strategy in enumerate(player_payoff):
                dominance = is_dominated(strategy, np.delete(player_payoff, index, 0))
                if strict and dominance[1]:
                    print('For player {} strategy {} is strictly dominated by strategy {} and will be removed\n'
                          .format(player, index, dominance[0]))
                    new_game = subset_game(game, index, player)
                    print('The reduced game is: ')
                    print(new_game)
                    print('********************************************************************************\n')
                    return iterated_elimination(new_game, strict)
                elif not(strict) and not(dominance[0] is None):
                    print('For player {} strategy {} is weakly dominated by strategy {} and will be removed\n'
                          .format(player, index, dominance[0]))
                    new_game = subset_game(game, index, player)
                    print('The reduced game is: ')
                    print(new_game)
                    print('********************************************************************************\n')
                    return iterated_elimination(new_game, strict)
        print('No more reduction possible for this game')
        return game

In [ ]:
game = domination_elimination_game()
game

In [ ]:
iterated_elimination(game)